In [3]:
from pycocotools.coco import COCO
from pycocotools import cocoeval

In [5]:
cocoDt = COCO("/home/pannuke_app/datasets/processed/CoNSeP/coco_annotations/test_annotations.json")

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [6]:
cocoGt = COCO("/home/pannuke_app/datasets/processed/CoNSeP/coco_annotations/test_annotations.json")

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [11]:
E = cocoeval.COCOeval(cocoGt,cocoDt); # initialize CocoEval object
# E.params.recThrs = ...;      # set parameters as desired
E.evaluate();                # run per image evaluation
E.accumulate();              # accumulate per image results
E.summarize();               # display summary metrics of results

Running per image evaluation...
Evaluate annotation type *segm*


KeyError: 'score'